<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/analysis/Stock_analysis%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
code = '092220'
start_date = '20000101'
freq = 'month' # freq: day, week, month

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import ast
import re

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_info(self):
    url = 'https://finance.naver.com/item/main.naver?code=' + code
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    cap = bs_obj.find_all('table', {'summary':'시가총액 정보'})[0].find('td').text
    cap = int(''.join(re.findall("\d+", cap)))
    pbr = float(bs_obj.find_all('table', {'summary':'PER/EPS 정보'})[0].find_all('em', {'id':"_pbr"})[0].text)
    per = bs_obj.find_all('table', {'summary':'PER/EPS 정보'})[0].find('td').text[:15]
    per = float(re.findall("\d+\.\d+", per)[0])
    sector_per = bs_obj.find_all('table', {'summary':'동일업종 PER 정보'})[0].find('td').text[:15]
    sector_per = float(re.findall("\d+\.\d+", sector_per)[0])
    return {'cap': cap, 'pbr': pbr, 'per': per, 'sector_per': sector_per}

  def get_price(self, start:int='20180501', interval='day'):
    url ='https://api.finance.naver.com/siseJson.naver?symbol=' + self.code + '&requestType=1&startTime=' \
    + start + '&endTime=' + str(datetime.datetime.now().date().strftime("%Y%m%d")) + '&timeframe=' + interval
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    b = bs_obj.get_text()
    for i in ['\n', '\t', "\\", ' ']:
      b = b.replace(i,'')

    data = np.array(ast.literal_eval(b)[1:])

    dic = {'Date':list(map(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'), data[:,0])), \
      'Open':np.array(data[:,1], float), 'High':np.array(data[:,2], float), 'Low':np.array(data[:,3], float),\
       'Close':np.array(data[:,4], float), 'Volume':np.array(data[:,5], float)}

    df = pd.DataFrame(data=dic)
    df = df.set_index('Date')
    df['Change'] = df['Close'][1:]/df['Close'][:-1].values
    df = df[1:]
    return df
    
  def get_fundamental(self):
    url = 'https://finance.naver.com/item/main.nhn?code=' + self.code
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    ths = bs_obj.find_all("th", {'scope':'col'})
    ths = [th.get_text() for th in ths][10:-22]
    dates = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),ths))
    dates = list(map(lambda x: x + '(Y)', dates[:4])) + list(map(lambda x: x + '(M)', dates[4:]))

    tr = bs_obj.find_all("tbody")
    tds = tr[2].find_all('td')
    tds = [td.get_text() for td in tds]
    elements = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),tds))
    elements = list(map(lambda x: float(x) if x.replace('.','').isdigit() else np.nan, elements))

    temp_dict = {}
    cnt = 0
    index = ['sales', 'operating profit', 'net income', 'operating margin', 'net margin', 'roe', 'debt ratio', 'quick ratio', \
    'reserve ratio', 'eps', 'per', 'bps', 'pbr', 'dividend per share', 'dividend yield ratio', 'dividend payout ratio']

    for i in dates:
      temp_dict[i] = elements[cnt::10]
      cnt += 1

    df = pd.DataFrame.from_dict(temp_dict)
    df = df.set_index([pd.Index(index)])
    return df

In [29]:
# stock
stock = Stock(code)
stock_df = stock.get_price(start_date, freq)

# market
market_df = Stock('KOSPI')
market_df = market_df.get_price(start_date, freq)

In [30]:
import matplotlib.pyplot as plt

class Analyser:
  def __init__(self, stock, market_df, freq):
    # Technical 
    self.df = stock.get_price(start_date, freq)
    # 주식 액면분할 날짜 제거
    self.market_df = market_df.drop(index=market_df.index.difference(self.df.index))
    self.risk_free = 1.025
    if freq == 'day': self.risk_free = self.risk_free**(1/250)
    elif freq == 'week': self.risk_free = self.risk_free**(1/50)
    elif freq == 'month': self.risk_free = self.risk_free**(1/12)

    # Fundamental
    self.funda = stock.get_fundamental()
    self.y_df = self.funda[list(filter(lambda x: 'Y' in x, self.funda))]
    self.m_df = self.funda[list(filter(lambda x: 'M' in x, self.funda))]

  # Technical 
  def plot_hist(self):
    plt.subplots(figsize=(8, 10))
    plt.hist(self.df['Change'], bins= 'auto', alpha=0.5)
    plt.hist(self.market_df['Change'], bins= 'auto', alpha=0.5)
    plt.legend(['Stock', 'Index'])
    plt.show()
  
  def get_moments(self):
    self.mean = round(self.df['Change'].mean(), 10)
    self.var = round(self.df['Change'].var(), 10)
    self.skew = round(self.df['Change'].skew(), 10)
    self.kurt = round(self.df['Change'].kurt(), 10)
    return self.mean, self.var, self.skew, self.kurt

  # Sharpe Ratio: Total risk
  def get_sharpe(self):
    self.sharpe_ratio = round(((self.df['Change'].mean() - self.risk_free) / self.df['Change'].std()), 10)
    return self.sharpe_ratio

  # Treynor Ratio: Market risk
  def get_treynor(self):
    cov = np.cov(np.array(self.df['Change']), np.array(self.market_df['Change']))[0][1]
    market_variance =  self.market_df['Change'].var()
    beta = cov/market_variance
    self.treynor_ratio = (self.df['Change'].mean() - self.risk_free)/beta
    return self.treynor_ratio
  
  # Sortino Ratio: Down risk
  def get_sortino(self):
    risk_free_min = self.risk_free
    downside_std = self.df[self.df['Change'] < 1]['Change'].std()
    self.sortino_ratio = (self.df['Change'].mean() - risk_free_min)/downside_std
    return self.sortino_ratio

  # Jensen's Alpha: Excess market
  def get_jensens(self):
    cov = np.cov(np.array(self.df['Change']), np.array(self.market_df['Change']))[0][1]
    market_variance =  self.market_df['Change'].var()
    beta = cov/market_variance
    self.jensens_alpha = self.df['Change'].mean() - (self.risk_free + beta*(self.market_df['Change'].mean() - self.risk_free))
    return self.jensens_alpha

  # Information Ratio: Persistence
  def get_information(self):
    benchmark_return = self.market_df['Change'].mean()
    tracking_error = (sum((self.df['Change']-self.market_df['Change'])**2)/(len(self.df.index)-1))**(1/2)
    self.information_ratio = (self.df['Change'].mean() - benchmark_return)/tracking_error
    return self.information_ratio

  # Fundamental
  def subplot_fundamental(self, li):
    fig, axs = plt.subplots(len(li), 2, gridspec_kw={'width_ratios': [4, 5]}, figsize=(20,10))
    fig.tight_layout(pad=3.0)
    for e in enumerate(li):
      for j in range(2):
        axs[e[0],j].title.set_text(e[1])
        if j == 0: 
          axs[e[0],j].plot(self.y_df.loc[e[1]])
          axs[e[0],j].axhline(self.y_df.loc[e[1]].mean(), color = 'r', linestyle = '--')
        elif j == 1: 
          axs[e[0],j].plot(self.m_df.loc[e[1]])
          axs[e[0],j].axhline(self.m_df.loc[e[1]].mean(), color = 'r', linestyle = '--')

In [31]:
analyser = Analyser(stock, market_df, freq)

In [32]:
i = stock.get_info()

In [33]:
print('시가총액: ', i['cap'], '억원')
print('PBR: ', i['pbr'])
print('PER: ', i['per'])
print('Sector PER: ', i['sector_per'])
print('PER/Sector PER: ', round(i['per']/i['sector_per'], 2))

# 섹터PER미만, PBR 1미만
if i['per']/i['sector_per'] > 1 or i['pbr'] > 1:
  print('='*50 + 'CUT' + '='*50)

시가총액:  3684 억원
PBR:  1.25
PER:  25.71
Sector PER:  7.31
PER/Sector PER:  3.52
==================================================CUT==================================================


In [40]:
analyser.y_df

,2019.12(Y),2020.12(Y),2021.12(Y),2022.12(E)(Y)
sales,2017.00,2010.00,2675.00,NaN
operating profit,NaN,NaN,265.00,NaN
net income,NaN,NaN,102.00,NaN
operating margin,NaN,NaN,9.92,NaN
net margin,NaN,NaN,3.80,NaN
roe,NaN,NaN,4.41,NaN
debt ratio,56.20,118.25,43.50,NaN
quick ratio,96.84,160.69,150.78,NaN
reserve ratio,151.99,77.23,194.55,NaN
eps,NaN,NaN,80.00,NaN


In [34]:
# PER/PBR 과거 2년보다 적을 것
if analyser.y_df.loc['pbr'][0] < i['pbr'] or analyser.y_df.loc['per'][0] < i['per']:
  print('='*50 + 'CUT' + '='*50)

==================================================CUT==================================================


In [35]:
# 분기별/연별 영업이익 +
if any(analyser.y_df.loc['operating profit'] > 0) and any(analyser.m_df.loc['operating profit'] > 0) == False:
  print('='*50 + 'CUT' + '='*50)

In [36]:
# 부채비율 100미만, 유보율 500이상
if (any(analyser.y_df.loc['debt ratio'] < 100) and any(analyser.m_df.loc['debt ratio'] < 100) == False) or \
(any(analyser.y_df.loc['reserve ratio'] > 500) and any(analyser.m_df.loc['reserve ratio'] > 500) == False):
  print('='*50 + 'CUT' + '='*50)


5. 근 1~2년 횡보 여부
6. 과거 10년 3배 넘긴 여부

In [46]:
url = 'https://finance.naver.com/item/main.nhn?code=' + code
result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
bs_obj = BeautifulSoup(result.content, "html.parser")

ths = bs_obj.find_all("th", {'scope':'col'})
ths = [th.get_text() for th in ths][10:-22]
dates = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),ths))
dates = list(map(lambda x: x + '(Y)', dates[:4])) + list(map(lambda x: x + '(M)', dates[4:]))

tr = bs_obj.find_all("tbody")
tds = tr[2].find_all('td')
tds = [td.get_text() for td in tds]
print(tds)
elements = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),tds))
print(elements)
elements = list(map(lambda x: float(x) if x.replace('.','').isdigit() else np.nan, elements))

temp_dict = {}
cnt = 0
index = ['sales', 'operating profit', 'net income', 'operating margin', 'net margin', 'roe', 'debt ratio', 'quick ratio', \
'reserve ratio', 'eps', 'per', 'bps', 'pbr', 'dividend per share', 'dividend yield ratio', 'dividend payout ratio']

for i in dates:
  temp_dict[i] = elements[cnt::10]
  cnt += 1

df = pd.DataFrame.from_dict(temp_dict)
df = df.set_index([pd.Index(index)])
df

['\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t2,017\n\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t2,010\n\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t2,675\n\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t\xa0\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t644\n\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t709\n\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t715\n\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t

IndexError: ignored

0. 시가총액
1. 현재 pbr
2. pbr 추이
3. 현재 per
4. 업종 per 비교, 추이

5. 시총대비 영업이익
6. 근 3년 영업이익 추이
7. 

5. 근 1~2년 횡보 여부
6. 과거 10년 2배 넘긴 여부
7. 등락폭